In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelpia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patroits",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dalls Cowboys"}

In [3]:
W=set(list(range(0,12))) #set of weeks, total of 12 weeks
T=set(list(range(0,24))) #set of teams all, total of 24 teams
 #set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
E=pd.read_csv("distance.csv",index_col=0)

In [4]:
# Create an empty model
m1= gp.Model(name="NFL1")

Using license file /Users/zhangmaojn/gurobi.lic
Academic license - for non-commercial use only


### Decision Variable & Objectives

In [5]:
home=len(E.columns)
away=len(E.columns)
weeks=12
#if team h plays at home against team j at week k
x = m1.addVars(home, away, weeks, vtype = GRB.BINARY, name = "x")

In [6]:
#if team h plays at home in week k and k+1
y = m1.addVars(home, weeks, vtype = GRB.BINARY, name = "y")

In [7]:
#if team h plays at away in week k and k+1
z = m1.addVars(home, weeks, vtype = GRB.BINARY, name = "z")

In [24]:
# Objective is to minimize the total distance of all teams
#obj = sum(sum(x[w,h,a]*2*E.iloc[h,a] for h in range(home) for a in range(away)) for w in range(weeks))
obj = sum(sum(x[h,a,w]*2*E.iloc[h,a] for w in range(weeks)) for h in range(home) for a in range(away))

### Constraints

In [25]:
#The season was limited to 12 weeks.
con1 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for w in W for h in T) == 12 
                     for a in T)

In [26]:
#each team would play once per week
con2 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for h in T) == 1 
                    for a in T for w in W)

In [27]:
#add constraints that all 12 games that a team played would need to be against a different opponent 
con3 = m1.addConstrs(sum(x[h,a,w] + x[a,h,w] for w in W) <= 1
                    for h in T for a in T)

In [28]:
#add constraints that each team would play at most 6 home games
con4 = m1.addConstrs(sum(x[h,a,w] for h in T for w in W) <= 6 for a in T)

In [29]:
#add constraints that no team can plays itself
con0 = m1.addConstrs((x[h,h,w] == 0 for h in T for w in W))

In [14]:
#add constraints that no team would play more than 2 consecutive games in home
#con5 = m1.addConstrs(sum(y[h,w] for w in W) <= 1 for h in T)

In [15]:
#Implication of constraint 5
#con5_i = m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] for a in T) <= 1 + y[h,w] for w in range(W-1) for h in T)

In [16]:
#add constraints that no team would play more than 2 consecutive games in away
#con6 = m1.addConstrs(sum(z[h,w] for w in range(weeks)) <= 1 for h in range(home))

In [17]:
#Implication of constraint 6
#con6_i = m1.addConstrs(sum(x[h,a,w] + x[h,a,w+1] for a in range(away)) <= 1 + z[h,w] for w in range(weeks-1) for h in range(home))

### Optimize

In [30]:
m1.setObjective(obj, GRB.MINIMIZE)
m1.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 3552 rows, 7488 columns and 122640 nonzeros
Model fingerprint: 0xc43529ae
Variable types: 0 continuous, 7488 integer (7488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 0 rows and 48 columns
Presolve time: 0.01s

Explored 0 nodes (0 simplex iterations) in 0.16 seconds
Thread count was 1 (of 4 available processors)

Solution count 0

Model is infeasible
Best objective -, best bound -, gap -
